# Real-Time Plotting of Arduino Data

**UNDER DEVELOPMENT!**

Based on YT video<br>
*_Create GUI apps in Jupyter using ipywidgets_ by **Everton Colling**

https://www.youtube.com/watch?v=f0WmLo8AVxo



In [2]:
# import libraries
from ipywidgets.widgets import Label, FloatProgress, FloatSlider, Button
from ipywidgets.widgets import Layout, HBox, VBox
from IPython.display import display
import bqplot as bq
import numpy as np
import time
import threading
import serial

In [15]:
# flag 
flag = True

x = np.linspace(0, 2*np.pi, 500)
dx = x[1] - x[0]
y = 1 + np.sin(x)

In [31]:
b_stop = Button(
    description = "Stop", 
    icon="stop",
    button_style="warning",
    layout = Layout(width="100px")
)

def stop_click(b):
    global flag
    flag = False
    
b_stop.on_click(stop_click)

w1=FloatProgress(
    value=y[-1],
    min=0,
    max=2,
    description="PV:",
    style={"description_width": "initial"},
    layout=Layout(width="365px")
)

w2=Label(
    value=str(np.round(y[-1],2)),
    layout = Layout(margin="0 10px 0 31px")
)

w12 = HBox(
    children=(w1, w2),
    layout=Layout(margin="0 0 0 43px")
)

wA = FloatSlider(
    value=0,
    min=0,
    max=0.5,
    step=0.01,
    description="Noise:",
    layout=Layout(width="490px", margin="0 0 5px 0")
)

In [33]:
# Plot elements
x_sc=bq.LinearScale()
y_sc=bq.LinearScale()

x_ax=bq.Axis(
    label="x(t)",
    scale=x_sc
)

y_ax=bq.Axis(
    label="y(t)",
    scale=y_sc,
    orientation = "vertical"
)

Line = bq.Lines(
    x=x,
    y=y,
    scales={"x": x_sc, "y": y_sc}
)

fig=bq.Figure(
    layout=Layout(width="500px", height="300px"),
    axes=[x_ax, y_ax],
    marks=[Line],
    fig_margin=dict(top=10, bottom=40, left=50, right=10)
)

#fig

Figure(axes=[Axis(label='x(t)', scale=LinearScale()), Axis(label='y(t)', orientation='vertical', scale=LinearS…

In [34]:
box = VBox(
    children=(fig, w12, wA),
    layout=Layout(border="solid 2px gray", width="510px")
)

app = VBox(
    children=(b_stop, box)
)

app

In [36]:
# Loop function
def work():
    global x
    global y
    
    while flag:
        A = wA.value
        x = np.delete(x,0)
        y = np.delete(y,0)
        
        x= np.append(x,x[-1]+dx)
        noise = A*np.random.rand()
        y = np.append(y, 1 + np.sin(x[-1]) + noise)
        
        w1.value = y[-1]
        w2.value = str(np.round(y[-1],2))
        
        Line.x = x
        Line.y = y
        
        time.sleep(0.05)

In [40]:
flag = True

thread = threading.Thread(target=work)

display(app)

thread.start()

In [41]:
thread.is_alive()

True